In [1]:
from scanbox.nlw_controller import *

box = BoxController('COM4')



In [3]:

# sbox_get_version(box.usb)
#         print('Version',struct.unpack(self.usb.read(3),'HHH'),flush=True)
        


In [9]:
print(sbox_get_version(box.usb))
sbox_set_lcd_token(box.usb,1)
sbox_set_optotune_active(box.usb,0)
sbox_set_current_power_active(box.usb,0)
sbox_set_status_message(box.usb)
# box.usb.close()

(1, 12, 0)


In [39]:
sbox_set_status_message(box.usb)

In [9]:
sbox_set_status_message(box.usb)

In [16]:
box.usb.read()

b''

In [2]:
box.set_continuous_resonant(True)


bytearray(b'4\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00')


In [3]:
box.set_linescan_mode('bidi')


bytearray(b'5\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00')


In [5]:
from scanbox.nlw_controller import box_commands
box.cmd_queue.put(box_commands['version'])


bytearray(b'x\x00\x00\x00\xaa\x00\x00\x00U\x00\x00\x00')
Has read.
b'\x01\x0c\x00'


<BoxController(Thread-6, stopped 8432)>